In [1]:
!pip install ultralytics==8.*

In [ ]:
# train 데이터 변환
# json파일의 x,y,w,h 바운딩박스의 값을 0~1의 값으로 변환하여 저장
# txt파일에는 class_id와 bbox의 값 x,y,w,h 값이 나오게 된다.
# 해당 과정을 통해 imgsize를 512에서 256으로 학습이 가능하게 됨
# 여러개의 bbox의 경우에는 줄바꿈으로 저장됨
# chi_id가 1~7이면 모두 굴뚝(class 0)으로 처리

import os
import json

input_dir = "TL_KS_BBOX"
output_dir = "labels/train"
os.makedirs(output_dir, exist_ok=True)

# chi_id 1~7 → class 0 (굴뚝)
CLASS_WHITELIST = {str(i) for i in range(1, 8)}  # {"1","2","3","4","5","6","7"}
CHIMNEY_CLASS_ID = 0 #굴뚝이 있다

for filename in os.listdir(input_dir):
    if not filename.endswith(".json"):
        continue

    json_path = os.path.join(input_dir, filename)
    txt_path = os.path.join(output_dir, filename.replace(".json", ".txt"))

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    lines = []
    # 최상위 키마다 entry 구조 순회
    for _, entry in data.items():
        fa = entry.get("file_attributes", {})
        try:
            img_w = int(float(fa.get("img_width", 0)))
            img_h = int(float(fa.get("img_height", 0)))
        except Exception:
            img_w, img_h = 0, 0
        if img_w <= 0 or img_h <= 0:
            # 이미지 크기 없으면 스킵
            continue

        for region in entry.get("regions", []):
            shape = region.get("shape_attributes", {})
            attrs = region.get("region_attributes", {})

            chi_id = attrs.get("chi_id")
            if chi_id is None:
                continue
            chi_id = str(chi_id).strip()

            # 1~7만 굴뚝으로 사용
            if chi_id not in CLASS_WHITELIST:
                continue

            # 좌표 읽기
            try:
                x = float(shape["x"])
                y = float(shape["y"])
                w = float(shape["width"])
                h = float(shape["height"])
            except Exception:
                continue

            if w <= 0 or h <= 0:
                continue

            # YOLO 정규화 (cx, cy, w, h)
            cx = (x + w/2) / img_w
            cy = (y + h/2) / img_h
            nw =  w / img_w
            nh =  h / img_h

            # 굴뚝 class 0으로 저장
            lines.append(f"{CHIMNEY_CLASS_ID} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    # 라벨 저장 (없으면 빈 파일 생성될 수 있음 → 원치 않으면 if lines: 로 감싸기)
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print(f"YOLO 변환 완료: {json_path} → {txt_path}")

print("작업종료")


In [ ]:
# valid 데이터 변환
# json파일의 x,y,w,h 바운딩박스의 값을 0~1의 값으로 변환하여 저장
# txt파일에는 class_id와 bbox의 값 x,y,w,h 값이 나오게 된다.
# 해당 과정을 통해 imgsize를 512에서 256으로 학습이 가능하게 됨
# 여러개의 bbox의 경우에는 줄바꿈으로 저장됨
# chi_id가 1~7이면 모두 굴뚝(class 0)으로 처리

import os
import json

input_dir = "VL_KS_BBOX_json"
output_dir = "labels/valid"
os.makedirs(output_dir, exist_ok=True)

# chi_id 1~7 → class 0 (굴뚝)
CLASS_WHITELIST = {str(i) for i in range(1, 8)}  # {"1","2","3","4","5","6","7"}
CHIMNEY_CLASS_ID = 0

for filename in os.listdir(input_dir):
    if not filename.endswith(".json"):
        continue

    json_path = os.path.join(input_dir, filename)
    txt_path = os.path.join(output_dir, filename.replace(".json", ".txt"))

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    lines = []
    # 최상위 키마다 entry 구조 순회
    for _, entry in data.items():
        fa = entry.get("file_attributes", {})
        try:
            img_w = int(float(fa.get("img_width", 0)))
            img_h = int(float(fa.get("img_height", 0)))
        except Exception:
            img_w, img_h = 0, 0
        if img_w <= 0 or img_h <= 0:
            # 이미지 크기 없으면 스킵
            continue

        for region in entry.get("regions", []):
            shape = region.get("shape_attributes", {})
            attrs = region.get("region_attributes", {})

            chi_id = attrs.get("chi_id")
            if chi_id is None:
                continue
            chi_id = str(chi_id).strip()

            # 1~7만 굴뚝으로 사용
            if chi_id not in CLASS_WHITELIST:
                continue

            # 좌표 읽기
            try:
                x = float(shape["x"])
                y = float(shape["y"])
                w = float(shape["width"])
                h = float(shape["height"])
            except Exception:
                continue

            if w <= 0 or h <= 0:
                continue

            # YOLO 정규화 (cx, cy, w, h)
            cx = (x + w/2) / img_w
            cy = (y + h/2) / img_h
            nw =  w / img_w
            nh =  h / img_h

            # 굴뚝 class 0으로 저장
            lines.append(f"{CHIMNEY_CLASS_ID} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    # 라벨 저장 (없으면 빈 파일 생성될 수 있음 → 원치 않으면 if lines: 로 감싸기)
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print(f"YOLO 변환 완료: {json_path} → {txt_path}")

print("작업종료")


In [ ]:
from ultralytics import YOLO

# 1) 모델 로드 (yolov8m)
model = YOLO("yolov8l.pt")

# 2) 학습
results = model.train(
    data="data.yaml",  # 데이터셋 yaml 경로
    conf=0.25,
    iou=0.5,
    max_det=100,
    epochs=100,
    imgsz=512,          # image size
    batch=16,
    workers=4,          # 로더 스레드 수
    project="runs",     # 결과 저장 (시각화)
    name="yolov8l_chimney",  # 실험명
    exist_ok=False,      # 덮어쓰기 허용
    verbose=True,
    patience=20,        # 20epoch 이후에도 성능 향상이 없을시 종료
    seed=42,
)

'''
data.yaml
train: images/train
val : images/valid

names:
  0: chimney
'''

# 3) 검증(선택: best.pt로 val set 평가)
metrics = model.val(data="data.yaml")

# 4) 예측(시각화 저장)
pred = model.predict(
    source="images/val",  # 검즘용 이미지 폴더
    conf=0.25,
    imgsz=512,
    save=True,
    project="runs",
    name="yolov8l_chimney_pred",
    exist_ok=True
)

# Map50 : 0.992

'''
폴더 형태
| mission1
  |--- TL_KS_BBOX (JSON 형태)
  |--- VL_KS_BBOX (JSON 형태)
  |--- images
    |--- trian (TS_KS)
    |--- valid (VS_KS)
  |--- labels
    |--- train (TL_KS_BBOX_TXT) #TXT형태
    |--- valid (VL_KS_BBOX_TXT) #TXT형태
  | mission1.ipynb
  | data.yaml
  
'''

New https://pypi.org/project/ultralytics/8.3.199 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.198  Python-3.11.13 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/mission1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=100, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_c

RuntimeError: Dataset '/content/drive/MyDrive/mission1/data.yaml' error  '/content/drive/MyDrive/mission1/data.yaml' does not exist